In [1]:
import pandas as pd
import pickle
import numpy as np
import xml.etree.cElementTree as ET
import os
import re
from tqdm import tqdm
import csv
import requests

In [ ]:
payment = pickle.load(open('./data/payment_dict.pkl', 'rb'))
trial2npi = pickle.load(open('./data/trial2npi.pkl', 'rb'))
trial_list = list(trial2npi.keys())

In [3]:
def process_trial(path):
    cur_trial = ET.ElementTree(file=path)

    if cur_trial.find('phase') == None:
        cur_phase = None
    else:
        cur_phase = cur_trial.find('phase').text
    
    return cur_phase    

In [4]:
def process_trial_enroll(path):
    cur_trial = ET.ElementTree(file=path)

    if cur_trial.find('enrollment') == None:
        cur_phase = None
    else:
        cur_phase = cur_trial.find('enrollment').text
    
    return cur_phase    

In [5]:
def process_trial_country(path):
    tree = ET.parse(path)
    root = tree.getroot()
    
    countries = []
    for country in root.findall('.//location_countries/country'):
        countries.append(country.text)
    
    return countries

In [ ]:
trial_dict = {}
country_cnt = {}
for path, subdirs, files in tqdm(os.walk('./data/AllPublicXML/')):
    for name in files:
        if name[:-4] in trial_list:
            cur_trial = os.path.join(path, name)
            cur_res = process_trial_country(cur_trial)
            trial_dict[name[:-4]] = cur_res
            for country in cur_res:
                if country not in country_cnt:
                    country_cnt[country] = 1
                else:
                    country_cnt[country] += 1
        else:
            continue


589it [00:39, 14.89it/s]


In [16]:
for each_trial in trial_dict:
    if 'United States' not in trial_dict[each_trial] and len(trial_dict[each_trial]) > 0:
        print(each_trial,trial_dict[each_trial])

NCT00259428 ['Belgium', 'Czech Republic', 'Denmark', 'Finland', 'France', 'Germany', 'Hungary', 'Italy', 'Netherlands', 'Poland', 'Spain', 'United Kingdom']
NCT00883337 ['Belgium', 'Canada', 'Czech Republic', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Poland', 'Spain', 'Switzerland', 'Tunisia', 'United Kingdom']
NCT01207440 ['Australia', 'Belgium', 'France', 'Germany', 'Italy', 'Korea, Republic of', 'Netherlands', 'Singapore', 'Spain', 'Sweden', 'United Kingdom']
NCT01344096 ['Israel']
NCT01374516 ['Brazil', 'Colombia', 'Honduras', 'Mexico', 'Puerto Rico']
NCT01385657 ['Australia', 'Germany', 'New Zealand']
NCT01424670 ['Estonia', 'Latvia', 'Lithuania', 'Moldova, Republic of', 'Peru', 'Philippines', 'South Africa']
NCT01469013 ['Japan']
NCT01487096 ['Canada', 'France']
NCT01548404 ['Czechia', 'France', 'Germany', 'Hungary', 'Poland']
NCT01576029 ['Brazil']
NCT01624233 ['Japan']
NCT01632163 ['China', 'India', 'Korea, Republic of', 'Russian Federation']
NCT01651637 ['United King

In [17]:
trial2npi['NCT00259428']

['1114922945', '1235515313', '1730137407', '1417976267', '1710945068']

In [ ]:
npi_info = pickle.load(open('./data/npi_info_dict.pkl', 'rb'))

In [21]:
for npi in npi_info:
    if npi_info[npi]['Country'] != 'United States':
        print(npi_info[npi]['Country'])

Canada
Canada
Chile
Canada
Argentina
Great Britain (Uk)
Great Britain (Uk)
Korea (Republic of)


In [ ]:
trial_dict = {}
for path, subdirs, files in tqdm(os.walk('./data/AllPublicXML/')):
    for name in files:
        if name[:-4] in trial_list:
            cur_trial = os.path.join(path, name)
            cur_res = process_trial(cur_trial)
            trial_dict[name[:-4]] = cur_res
        else:
            continue


589it [00:33, 17.47it/s]


In [ ]:
pickle.dump(trial_dict, open('./data/trial_phase.pkl', 'wb'))

In [ ]:
trial_dict = {}
for path, subdirs, files in tqdm(os.walk('./data/AllPublicXML/')):
    for name in files:
        if name[:-4] in trial_list:
            cur_trial = os.path.join(path, name)
            cur_res = process_trial_enroll(cur_trial)
            trial_dict[name[:-4]] = cur_res
        else:
            continue

589it [00:42, 13.94it/s]


In [ ]:
trial_info = pickle.load(open('./data/trial_info.pkl', 'rb'))

In [ ]:
pickle.dump(trial_dict, open('./data/trial_enroll.pkl', 'wb'))

In [7]:
def get_icd_category(icd_code):
    # C00-D49: Oncology
    # I00-I99: Cardiology
    # G00-G99 : Neurology
    # E00-E89 : Endocrinology
    # A00-B99 : Infectious Disease
    first_three = icd_code[:3]
    if first_three >= 'C00' and first_three <= 'D49':
        return 'Oncology'
    elif first_three >= 'I00' and first_three <= 'I99':
        return 'Cardiology'
    elif first_three >= 'G00' and first_three <= 'G99':
        return 'Neurology'
    elif first_three >= 'E00' and first_three <= 'E89':
        return 'Endocrinology'
    elif first_three >= 'A00' and first_three <= 'B99':
        return 'Infectious Disease'
    else:
        return 'Other'

In [16]:
trial2category = {}
for each_trial in trial_info:
    cur_cond = trial_info[each_trial]['condition']
    flag = False
    for each_cond in cur_cond:
        cur_cate = get_icd_category(each_cond)
        if cur_cate != 'Other':
            trial2category[each_trial] = cur_cate
            flag = True
            break
# Get statistics
category_count = {}
for each_trial in trial2category:
    cur_cate = trial2category[each_trial]
    if cur_cate in category_count:
        category_count[cur_cate] += 1
    else:
        category_count[cur_cate] = 1
    

In [ ]:
category_count
pickle.dump(trial2category, open('./data/trial2category.pkl', 'wb'))